In [1]:
# Импорт всех необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = pd.read_excel('Данные_для_курсовои_Классическое_МО.xlsx').drop(columns=['Unnamed: 0'])
data.dropna(inplace=True)

# Переместим названия колонок в отдельную переменную, пригодится
columns = data.columns.tolist()

# Быстрая проверка
data.head()

#'IC50, mM', 'CC50, mM', 'SI'

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop(columns=['IC50, mM','CC50, mM','SI'])
y = data['CC50, mM']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred_lin = linreg.predict(X_test_scaled)
print("Linear Regression R2:", r2_score(y_test, y_pred_lin))

Linear Regression R2: -36381.65494503056


In [6]:
rf = RandomForestRegressor(random_state=42)
param_grid_rf = {'n_estimators':[100], 'max_depth':[5,10]}
grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("Random Forest R2:", r2_score(y_test, y_pred_rf))

Random Forest R2: 0.4522144833360827


In [7]:

svr = SVR()
param_grid_svr = {'C':[10,100], 'gamma':['scale'], 'epsilon':[0.1,0.5]}
grid_svr = GridSearchCV(svr, param_grid_svr, cv=3, scoring='neg_mean_squared_error')
grid_svr.fit(X_train_scaled, y_train)
best_svr = grid_svr.best_estimator_
y_pred_svr = best_svr.predict(X_test_scaled)
print("SVR R2:", r2_score(y_test, y_pred_svr))

SVR R2: 0.2550384472743574


In [8]:
results = pd.DataFrame({
    'Модель': ['Linear Regression', 'Random Forest', 'SVR'],
    'MAE': [
        mean_absolute_error(y_test, y_pred_lin),
        mean_absolute_error(y_test, y_pred_rf),
        mean_absolute_error(y_test, y_pred_svr)
    ],
    'MSE': [
        mean_squared_error(y_test, y_pred_lin),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, y_pred_svr)
    ],
    'R2': [
        r2_score(y_test, y_pred_lin),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, y_pred_svr)
    ]
})

display(results)

,Модель,MAE,MSE,R2
0,Linear Regression,9675.622306,1.713905e+10,-36381.654945
1,Random Forest,300.990410,2.580495e+05,0.452214
2,SVR,343.829193,3.509347e+05,0.255038


---

Linear Regression
MAE и MSE огромные: ошибки в десятки тысяч, что говорит о том, что линейная модель абсолютно не справляется с этой задачей.

Отрицательный R² (-36381.65): означает, что модель хуже, чем просто усреднение (модель не описывает зависимость вообще).

Почему?

Вероятно, что зависимость между признаками и CC50 нелинейная.

Линейная регрессия не может учитывать сложные взаимодействия между признаками.

---
Random Forest
MAE: 300.99 — сильно ниже, чем у линейной.

MSE: 2.58e+05 — в пределах разумных ошибок.

R²: 0.4522 — модель объясняет ≈ 45% дисперсии.

Вывод:
Random Forest работает намного лучше — он умеет учитывать сложные, нелинейные взаимодействия между признаками.

---
SVR
MAE: 343.83 — немного хуже, чем у Random Forest.

MSE: 3.50e+05 — больше, чем у RF.

R²: 0.2550 — объясняет только ≈ 25% дисперсии.

Вывод:
SVR показывает умеренный результат, но уступает Random Forest.

---

Общие выводы:
Задача прогноза CC50 требует нелинейных моделей.

Random Forest — на данном этапе показывает наилучшее качество.

SVR справляется хуже.

Линейная регрессия — практически не применима в этой задаче.

